# Manipulação de dados com SQL 🦑

![SQL](../img/aula02/sql_2.png)

## Revisão Aula 01

![Grupos SQL](../img/aula01/grupos_sql.png)

Os tipos da linguagem SQL são:

- **DQL - Data Query Language** - Linguagem de Consulta de dados.
    - São os comandos de consulta. Exemplos: `SELECT`, `FROM`
- **DML - Data Manipulation Language** - Linguagem de Manipulação de Dados.
    - São os comandos que interagem com os dados dentro das tabelas. Exemplos: `INSERT`, `DELETE`, e `UPDATE`.

### Ordem de execução dos comandos SQL

O SQL Server não faz o processamento dos dados na ordem de escrita, internamente o SQL Server segue a seguinte ordem:

![Execution Order](../img/aula01/execution_order.png)

## Funções de agregação 🍹

Para obter agregações, a partir de dados agregados, como a contagem de todos os registros pertencentes a uma categoria, precisamos utilizar a instrução `GROUP BY` combinada com uma das funções de agregação, `SUM()`, `COUNT()`, `AVG()`, `MAX()`, `MIN()` _etc_.

Funções SQL Server: [Aggregate Functions (Transact-SQL) - SQL Server | Microsoft Learn](https:\learn.microsoft.com\en-us\sql\t-sql\functions\aggregate-functions-transact-sql?view=sql-server-ver16)

In [ ]:
USE [SQL_TREINAMENTO]
GO

In [ ]:
-- Contagem de pontos cadastrados por matriz
SELECT
    zmatrix        AS [Matriz]
    ,COUNT(ID)     AS [Qtde Pontos]
FROM station
-- GROUP BY [Matriz]       -- Syntax error
GROUP BY [zmatrix]
-- ORDER BY COUNT(ID)
-- ORDER BY [Qtde Pontos]

In [ ]:
-- Estatísticas do NA subterrâneo por ponto
SELECT
    Station
    ,COUNT(Station)         AS [Medições]
    ,MIN(zwl_elevation)     AS [NA Min]
    ,AVG(zwl_elevation)     AS [NA Médio]
    ,MAX(zwl_elevation)     AS [NA Max]
FROM gw_level
GROUP BY Station

In [ ]:
-- Estatísticas do NA subterrâneo por ponto e ano
SELECT
    Station
    ,YEAR(date_)                      AS [Ano]
    ,COUNT(Station)                   AS [Medições]
    ,ROUND(MIN(zwl_elevation), 2)     AS [NA Mínimo]
    ,ROUND(AVG(zwl_elevation), 2)     AS [NA Médio]
    ,ROUND(MAX(zwl_elevation), 2)     AS [NA Máximo]
FROM gw_level
WHERE
    Station IN (88, 89, 93, 203, 208)
GROUP BY Station, YEAR(date_)
ORDER BY Station, [Ano]

Para aplicar filtros aos grupos agregados, não é possível utilizar o operador `WHERE`. Para isto, devemos usar a instrução `HAVING`.

> Ex.: Retornar as mesmas estatísticas anteriores, apenas para grupos com mais de 10 observações...

In [ ]:
SELECT
    Station
    ,YEAR(date_)                      AS [Ano]
    ,COUNT(Station)                   AS [Medições]
    ,ROUND(MIN(zwl_elevation), 2)     AS [NA Mínimo]
    ,ROUND(AVG(zwl_elevation), 2)     AS [NA Médio]
    ,ROUND(MAX(zwl_elevation), 2)     AS [NA Máximo]
FROM gw_level
WHERE
    Station IN (88, 89, 93, 203, 208)
    AND COUNT(Station) > 10
GROUP BY Station, YEAR(date_)
-- HAVING COUNT(Station) > 10
ORDER BY Station, [Ano]

## Mesclar múltiplas tabelas - JOINs 🤺

Para unir várias tabelas e trazer colunas adicionais, faremos uso do operador `JOIN`, na instrução `FROM`.

A sintaxe básica é: `FROM <tabela_A> [LEFT/RIGHT/INNER/FULL] JOIN <tabela_B> ON <tabela_A.coluna_A> = <tabela_B.coluna_B>`

> **DICA:** Quando se executa um `LEFT/RIGHT JOIN` e são retornados mais registros do que haviam inicialmente na tabela *left* ou *right*, possivelmente há registros duplicados (**de acordo com o critério utilizado no JOIN**).

![SQL Join Image](../img/aula02/join_types0.jpg "SQL Join image")

![SQL Join Image](../img/aula02/join_types.png "SQL Join image")

In [ ]:
-- Mesmo agrupamento anterior, apenas para os pontos da "Mina B"

SELECT TOP 100 *
FROM gw_level

[...]

In [ ]:
SELECT TOP 5 * FROM station
SELECT TOP 5 * FROM gw_level

In [ ]:
SELECT 
    vt.Name AS [TableName]
    ,vf.Name AS [FieldName]
    ,vf.[Description]
FROM ViewField vf
    JOIN ViewTable vt
        ON vf.ViewTable_Id = vt.Id
WHERE
    (vt.Name = 'station' AND vf.Name = 'ID')
    OR (vt.Name = 'gw_level' AND vf.Name = 'Station')

In [ ]:
-- Referenciar tabelas e colunas correspondentes

SELECT TOP 100
    -- *
    -- [zdata_source]
    station.[Name]
    ,station.[zlocation]
    ,station.[stn_type]
    ,gw_level.[date_]
    ,gw_level.[zwl_elevation]
    ,gw_level.[depth_]
FROM gw_level
    JOIN station    -- INNER
        ON gw_level.Station = station.ID

In [ ]:
SELECT
    s.[Name]                                AS [Codigo HGA]
    ,YEAR(gw.date_)                         AS [Ano]
    ,COUNT(gw.Station)                      AS [Medições]
    ,ROUND(MIN(gw.zwl_elevation), 2)        AS [NA Mínimo]
    ,ROUND(AVG(gw.zwl_elevation), 2)        AS [NA Médio]
    ,ROUND(MAX(gw.zwl_elevation), 2)        AS [NA Máximo]
FROM gw_level gw
    INNER JOIN station s
        ON gw.Station = s.ID
WHERE s.zlocation = 'Mina B'
GROUP BY s.[Name], YEAR(gw.date_)

> **DICA:** Quando não se conhece bem a modelagem e a relação entre as tabelas, pode-se utilizar alguma ferramenta de _Query Builder_ (SSMS, Excel, Power BI, **HGA** _etc_).

## `INNER JOIN` _vs_ `LEFT JOIN`

In [ ]:
-- INNER JOIN
SELECT COUNT(s.ID) AS [Qtde]
FROM gw_level gw 
    INNER JOIN station s
        ON gw.Station = s.ID

In [ ]:
-- LEFT JOIN: gw_level + station
SELECT COUNT(s.ID) AS [Qtde]
FROM gw_level gw 
    LEFT JOIN station s
        ON gw.Station = s.ID

In [ ]:
-- LEFT JOIN: station + gw_level
SELECT COUNT(s.ID) AS [Qtde]
FROM station s
    LEFT JOIN gw_level gw
        ON s.ID = gw.Station
-- WHERE gw.Station IS NULL

In [ ]:
SELECT TOP 100
    s.Name          AS [Codigo HGA]
    ,l.from_        AS [De (m)]
    ,l.to_          AS [Ate (m)]
    ,l.soil_type    AS [Litologia]
FROM station s
    LEFT JOIN lithology l
        ON s.ID = l.Station
WHERE s.zmatrix = 'Agua Subterranea'

In [ ]:
SELECT
    s.Name          AS [Codigo HGA]
    ,s.stn_type     AS [Tipo Ponto]
    ,s.zcomplex     AS [Complexo]
FROM station s
    LEFT JOIN lithology l
        ON s.ID = l.Station
WHERE s.stn_type LIKE 'Poco%'
    AND l.soil_type IS NULL

## "Concatenando" tabelas

Para anexar tabelas contendo os mesmos campos, podemos utilizar o operador `UNION` entre vários `SELECT <coluna_1>...<column_name_n> FROM <tabela>`.

> **DICA:** `UNION` reúne apenas linhas distintas entre as tabelas (remove duplicidades), enquanto `UNION ALL` reúne todas as linhas entre as tabelas, incluindo as duplicadas.

In [ ]:
-- Retornar todos os pontos das minas A e D
SELECT 
    [Name]
    ,zmatrix            AS [Matriz]
    ,stn_type           AS [Tipo Ponto]
    ,zlocation          AS [Local]
FROM station
WHERE [zlocation] = 'Mina A'

UNION
-- UNION ALL

SELECT 
    [Name]
    ,zmatrix            AS [Matriz]
    ,stn_type           AS [Tipo Ponto]
    ,zlocation          AS [Local]
    -- ,zcomplex           AS [Complexo]
FROM station
WHERE [zlocation] = 'Mina D'

In [ ]:
-- Retornar todos os pontos das minas A e D
SELECT 
    [Name]
    ,zmatrix            AS [Matriz]
    ,stn_type           AS [Tipo Ponto]
    ,zlocation          AS [Local]
FROM station
WHERE [zlocation] IN ('Mina A', 'Mina D')

# Subqueries 🤹‍♀️

Semelhante às **tabelas temporárias**, ***subqueries*** são uma maneira alternativa de armazenar resultados temporários de suas consultas para executar novas consultas na sequência.

Para criar uma **_subquery_**, basta criar uma segunda estrutura `SELECT ... FROM ...`, dentro da cláusula `...FROM (...) ...` ou `... WHERE (...) ...` da primeira consulta.

> **DICA:** **Subconsultas** às vezes são mais difíceis de ler e entender do que **tabelas temporárias**. O uso de tabelas **temporárias** permite separar sua lógica para criar a tabela em um formato ordenado intuitivamente.

## Exercício: Obter resultados de NA do ponto 'WST-88' acima da média

In [ ]:
-- Primeiro passo: Obter a média do ponto
SELECT AVG(gw.zwl_elevation)        AS [NA Médio]
FROM gw_level gw
    INNER JOIN station s
        ON gw.Station = s.ID
WHERE s.[Name] = 'WST-88'

-- Segundo passo: Selecionar novamente o conjunto de resultados,
-- utilizando as médias calculadas
SELECT TOP 100
    s.[Name]            AS [Codigo HGA]
    ,s.[stn_type]       AS [Tipo Ponto]
    ,gw.date_           AS [Data]
    ,gw.zwl_elevation   AS [Cota NA]
FROM gw_level gw
    INNER JOIN station s
        ON gw.Station = s.ID
WHERE zwl_elevation > (
    SELECT AVG(gw.zwl_elevation)        AS [NA Médio]
    FROM gw_level gw
        INNER JOIN station s
            ON gw.Station = s.ID
    WHERE s.[Name] = 'WST-88'
)

### 🧠 **Pré-Desafio**:

> ### Como seria possível obter os resultados de NA acima da média para os pontos 'WST-88' e 'WST-205' em uma única consulta ??

Obs.: Não vale usar `UNION`/`UNION ALL` 😒

******

## Tabelas temporárias

### Tipos de tabelas temporárias 🍺🍷🍹

**Tabelas temporárias** são úteis para consultas mais complicadas em que você precisa armazenar os resultados de uma transformação de tabela de forma intermediária para executar outra transformação na sequência.

Existem três tipos de tabelas temporárias no SQL que poderá criar:
1. Tabela temporária local
2. ***Common Table Expression* - CTE**
3. Tabela temporária global

#### i. Tabela temporária local
Quando você cria uma tabela temporária **local**, ela será armazenanda temporariamente em sua sessão SQL atual. Dessa forma, podemos executar o trecho do código que cria a **tabela temporária local**, e em seguida, consultar os resultados e realizar diferentes operações de transformação neles depois.

**Importante:** Quando criamos uma tabela **local**, seus resultados podem ser consultados apenas no mesmo painel onde o código para criá-la foi escrito.

Para criar uma tabela temporária **local**, utilizamos a instrução `INTO #<temp_table_name>` logo antes de escrever a parte `FROM <table_name>...`.

> **DICA:** Uma tabela temporária **local** não pode ser substituída, portanto, executar a consulta de criá-la novamente resultará em um erro. Portanto, a tabela precisa ser excluída antes de reexecutar a consulta. Uma maneira sucinta de evitar a criação de tabelas temporárias **locais** e, em seguida excluí-las, é incluir a seguinte linha no topo do seu código: `IF OBJECT_ID('tempdb..#<tabela>') IS NOT NULL DROP TABLE #<tabela>`

In [ ]:
-- Tabela temporária local

IF OBJECT_ID('tempdb..#na_medio') IS NOT NULL DROP TABLE #na_medio
GO

SELECT AVG(gw.zwl_elevation)        AS [NA Médio]
INTO #na_medio
FROM gw_level gw
    INNER JOIN station s
        ON gw.Station = s.ID
WHERE s.[Name] = 'WST-88'

-- Segundo passo: Selecionar novamente o conjunto de resultados,
-- utilizando as médias calculadas
SELECT TOP 100
    s.[Name]            AS [Codigo HGA]
    ,s.[stn_type]       AS [Tipo Ponto]
    ,gw.date_           AS [Data]
    ,gw.zwl_elevation   AS [Cota NA]
FROM gw_level gw
    INNER JOIN station s
        ON gw.Station = s.ID
WHERE zwl_elevation > (SELECT * FROM #na_medio)

#### ii. Common Table Expression (CTE)
Ao criar uma **CTE**, ela é armazenada temporariamente na **consulta atual**. Isso significa que os resultados da consulta não podem ser obtidos em uma parte posterior do código. Como a **CTE** não existe fora da consulta atual, não há necessidade de excluí-la explicitamente quando for necessário criá-la novamente.

Para criar uma **CTE**, envolva sua instrução `SELECT...` entre parênteses do seguinte código, `WITH <cte_name> AS ()...`

> **DICA:** À primeira vista, **CTE**s podem parecer inferiores às **tabelas temporárias locais**, mas elas são de fato muito poderosas. Devido à natureza em que eles só existem dentro da própria execução da consulta, eles podem ser usados na definição de **Views**, para inserir em uma tabela criada e como parte da criação de outros objetos SQL. **Tabelas temporárias locais** tipicamente não podem.

In [ ]:
WITH na_medio AS (
    SELECT AVG(gw.zwl_elevation)        AS [NA Médio]
    FROM gw_level gw
        INNER JOIN station s
            ON gw.Station = s.ID
    WHERE s.[Name] = 'WST-88'
)

SELECT TOP 100
    s.[Name]            AS [Codigo HGA]
    ,s.[stn_type]       AS [Tipo Ponto]
    ,gw.date_           AS [Data]
    ,gw.zwl_elevation   AS [Cota NA]
FROM gw_level gw
    INNER JOIN station s
        ON gw.Station = s.ID
WHERE zwl_elevation > (SELECT * FROM na_medio)

#### iii. Tabela temporária global
Quando uma tabela temporária **global** é criada, ela é armazenada *permanentemente* no banco de dados, `[tempdb]`, permitindo que outros usuários do mesmo servidor possam acessá-la Em contraste com as outras duas tabelas temporárias (**tabelas temporárias locais** e **CTEs**), você precisa excluir explicitamente as tabelas globais.

Criar uma tabela temporária **global** é como criar uma tabela temporária **local**, basta usar a instrução `INTO <##temp_table_name>` logo antes de escrever a parte `FROM <###table_name>...`. Observe a ênfase em dois símbolos hash, `##`, em vez de apenas um que é usado para **tabelas temporárias locais**.

> **AVISO:** O banco de dados `[tempdb]` armazena essas tabelas temporárias **global***, e como esse banco de dados é um pouco invisível, pois está oculto no *Object Explorer* no SSMS, muitas vezes pode-se esquecer de excluir a tabela depois para economizar espaço. Por isso a criação uma tabela temporária **global** é raramente aconselhável. 

## Desafio: Dominando subqueries e CTE's

Notebook completo: 📒 [Desafio Aula02](../desafios/aula02/Aula02_Challenge.ipynb)